In [1]:
%%file myConv.py

import torch
import numpy as np
def myTranspConv2d(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #всегда 1 шаг
    stride = 1

    #добавление отступов и padding в входной матрице
    pad = kernel_size - 1
    result_matrix = []
    for matr in matrix:
      zero_tensor = np.zeros((((matr.shape[0]-1)*(transp_stride)+1), ((matr.shape[1]-1)*(transp_stride)+1)))
      for a in range (0, zero_tensor.shape[0], transp_stride):
        for b in range (0, zero_tensor.shape[1], transp_stride):
          zero_tensor[a][b] = matr[a//(transp_stride)][b//(transp_stride)]

      pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
      result_matrix.append(pad_matr)
    matrix = torch.tensor(result_matrix)

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

    #инвертирование ядра для ConvTranspose2d
    filter_for_transpose = []
    for j in range(out_channels):
      filter_in = []
      for i in range(in_channels):
        filter_in.append(np.flip(np.array(filter[j][i])))
      filter_for_transpose.append(filter_in)

    filter_for_transpose = torch.tensor(filter_for_transpose)
    filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)



    spisok = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels):
            val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias
      spisok.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))


    return np.array(spisok), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_val))

  return Svertca

Overwriting myConv.py


In [2]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Overwriting conftest.py


In [5]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myTranspConv2d

tensor1 = torch.rand(3, 5, 6)
tensor2 = torch.rand(1, 28, 28)
tensor3 = torch.rand(7, 10, 10)


def test_1(session):
  myFunction = myTranspConv2d(in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, bias=True,)
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
  torchFunction.weight.data = kernel
  torchFunction.bias.data = bias_val
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myTranspConv2d(in_channels=1, out_channels=2, kernel_size=4, transp_stride=3, bias=True)
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=4, stride=3, bias=True)
  torchFunction2.weight.data = kernel2
  torchFunction2.bias.data = bias_val2
  myResult2 = str(np.round(result2, 2))
  torchResult2 = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult2 == myResult2


def test_3(session):
  myFunction3 = myTranspConv2d(in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, bias=True)
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.ConvTranspose2d(in_channels=7, out_channels=1, kernel_size=3, stride=5, bias=True)
  torchFunction3.weight.data = kernel3
  torchFunction3.bias.data = bias_val3
  myResult3 = str(np.round(result3, 2))
  torchResult3 = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult3 == myResult3


Overwriting test_http.py


In [6]:
!pytest

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.3, pluggy-1.3.0
rootdir: /content
plugins: anyio-3.7.1
collected 3 items                                                                                  

test_http.py ...                                                                             [100%]

========================================= warnings summary =========================================
test_http.py::test_1
  /content/myConv.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
    matrix = torch.tensor(result_matrix)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
=================================== 3 passed, 1 warning in 4.85s =============

In [7]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myTranspConv2d

tensor1 = torch.rand(1, 4, 4)

myFunction = myTranspConv2d(in_channels=1, out_channels=2, kernel_size=3, transp_stride=1, bias=True)
result, kernel, bias_val = myFunction(tensor1)

torchFunction = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, bias=True)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[1.05574824 1.44801757 1.45391226 1.43369135 1.02989569 0.85481453]
  [1.37833494 2.33189896 2.40381805 2.54183989 1.55330905 1.21388571]
  [1.80990752 2.95656571 2.9810659  2.80845913 1.51349075 1.08177028]
  [1.82686799 2.74422595 2.901886   2.15184225 1.44749381 0.94866295]
  [1.54828121 2.21929326 2.16829568 1.7689403  1.07839444 0.86061603]
  [0.98358241 1.3952758  1.22868066 0.99043887 0.84881538 0.81259815]]

 [[0.59956723 0.93334016 1.13434359 1.29887494 0.87414825 0.7104379 ]
  [0.96851506 2.02794454 2.57935446 3.0274901  1.75198392 1.24656206]
  [1.14651136 2.1508452  2.80433638 2.82055417 1.63477542 1.02517498]
  [1.30683096 2.12278085 2.91100677 2.40209178 1.48460615 0.71442036]
  [0.83491676 1.57496764 1.8289732  1.66609357 0.87942395 0.51710088]
  [0.46635226 0.64000076 0.67569497 0.64661608 0.47178963 0.41865492]]]
[[[1.0557482  1.4480176  1.4539123  1.4336914  1.0298957  0.8548145 ]
  [1.378335   2.331899   2.4038181  2.54184    1.5533091  1.2138858 ]
  [1.8099076  2.

/content/myConv.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  matrix = torch.tensor(result_matrix)


In [8]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myTranspConv2d

tensor2 = torch.rand(3, 5, 6)

myFunction2 = myTranspConv2d(in_channels=3, out_channels=1, kernel_size=2, transp_stride=2, bias=True)
result, kernel, bias_val = myFunction2(tensor2)

torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=2, stride=2, bias=True)
torchFunction2.weight.data = kernel
torchFunction2.bias.data = bias_val

print(result)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[1.55279903 1.60265495 1.31169631 1.70783476 1.19822481 1.73030863
   1.49029723 2.24083239 1.6759321  2.16668405 1.18335226 1.61907672]
  [1.55251734 1.94953043 1.77199189 1.63435051 1.90533006 1.53263803
   2.39512527 2.02820552 2.1689033  2.21793861 1.75901136 1.47528426]
  [1.05977008 1.52603855 1.17555134 1.3623931  1.01122597 1.0717073
   1.1671245  1.82531106 1.51305553 1.93736976 1.32457017 1.97841762]
  [1.63834576 1.23854646 1.4536338  1.41506529 1.09193364 1.06763204
   1.91150575 1.40819851 1.91011842 1.90897593 2.1130548  1.72166126]
  [1.60900759 1.8229656  1.33683107 1.72688515 1.51354566 2.00930302
   1.57034275 2.29497253 1.63039323 2.01215    1.19962487 2.01738018]
  [1.84096028 2.10777671 1.82954413 1.70975906 2.1095024  2.01728151
   2.42508455 2.14651186 2.02028919 2.14099983 2.21266939 1.55911172]
  [1.25656282 1.55370407 1.10773094 1.25743056 1.51823895 2.00380233
   1.68821117 2.29650253 1.30518959 1.34400826 1.55276241 1.57120157]
  [1.67409268 1.58270619 1.2

In [9]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myTranspConv2d

tensor3 = torch.rand(1, 4, 4)

myFunction3 = myTranspConv2d(in_channels=1, out_channels=1, kernel_size=1, transp_stride=4, bias=True)
result, kernel, bias_val = myFunction3(tensor3)

torchFunction3 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=4, bias=True)
torchFunction3.weight.data = kernel
torchFunction3.bias.data = bias_val

print(result)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[1.7578795  0.9843809  0.9843809  0.9843809  1.69221192 0.9843809
   0.9843809  0.9843809  1.29492853 0.9843809  0.9843809  0.9843809
   1.06375902]
  [0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809 ]
  [0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809 ]
  [0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809 ]
  [1.15466536 0.9843809  0.9843809  0.9843809  1.84756007 0.9843809
   0.9843809  0.9843809  1.60973307 0.9843809  0.9843809  0.9843809
   1.28181982]
  [0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809 ]
  [0.9843809  0.9843809  0.9843809  0.9843809  0.9843809  0.9843809
   0.9843809  0.9843809  0